# Fashion MNIST Image Classification - Azure ML SDK Training

In this Fashion MNIST notebook we introduce how to instrument your training process with the Azure ML SDK. 

This code will show how Azure ML SDK can support your machine learning project with:
* A central repository for your machine learning project
* Creating a cloud computer target and running your training in the cloud
* Whilst running your training in the cloud, add logging to the code to see in real time in your notebook the outputs and progress of the training on the remote compute in the cloud
* Saving your large datasets to azure storage so your training models can mount the data to the assigned training compute and have a 'one-source-of-truth' dataset for all your data science team to be using
* Finally registering this model with versioning so others can leverage it easily

This notebook is based off the great sample in the docs here: [https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml?WT.mc_id=aisummit-github-amynic)

## Import packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.2


Load your Azure ML workspace from the config file you setup. The config file will look like below:

![Config File](./images/configfile.JPG "Config file")

## Load the workspace created

In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: /data/home/amyboyd/notebooks/deeplearningfashion/azureml-sdk-fashion/config.json
amyboyd-ga-ws	westeurope	deeplearning	westeurope


In [3]:
experiment_name = 'deeplearning_fashion'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)
print(exp)

Experiment(Name: deeplearning_fashion,
Workspace: amyboyd-ga-ws)


## Create Cloud Compute Target 

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 2)

# This example uses GPU VM. For using CPU VM, set SKU to STANDARD_D2_V2
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

found compute target. just use it. gpucluster


In [5]:
import os
script_folder = './keras-fashion'
os.makedirs(script_folder, exist_ok=True)

## Upload and use data in an Azure File Store

In [6]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

# On first use uncomment the line below to upload your data
#ds.upload(src_dir='./data', target_path='fashiondata', overwrite=True, show_progress=True)

AzureBlob amyboydgaws1145015247 azureml-blobstore-ba93ae49-0766-40b8-9dd5-9b0da138eaa7


## Write train.py file and save model.h5

In [7]:
%%writefile $script_folder/train.py

import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
from sklearn.externals import joblib

import azureml
from azureml.core import Workspace, Run

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

data_folder = os.path.join(args.data_folder, 'keras-fashion')
print('Data folder:', data_folder)

#variables
num_classes = 10
batch_size = 128
epochs = 24
img_rows,img_cols = 28,28

#data for train and testing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9


print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, sep = '\n')

#data pre-processing
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test,  num_classes)

#formatting issues for depth of image (greyscale = 1) with different kernels (tensorflow, cntk, etc)
if K.image_data_format()== 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols,1)
    
    
    
#Define the CNN model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), padding = 'same', activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()



# get hold of the current run
run = Run.get_submitted_run()

print('Train a deep learning model')
model.compile(loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(), metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

#evaluate the model on the test data
print('Predict the test set')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

# calculate accuracy on the prediction
print('Accuracy is', score[1])

run.log('accuracy', np.float(score[1]))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
model.save('outputs/model.h5')


Overwriting ./keras-fashion/train.py


## Run the model in the cloud for training

In [8]:
import shutil
shutil.copy('utils.py', script_folder)

'./keras-fashion/utils.py'

In [9]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.as_mount()
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['keras', 'scikit-learn'])

In [14]:
#run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
deeplearning_fashion,deeplearning_fashion_1544876526872,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [18]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [15]:
run.wait_for_completion(show_output=True) # specify True for a verbose log

RunId: deeplearning_fashion_1544876526872

Streaming azureml-logs/80_driver_log.txt

tensorflow Version is: 1.12.0
Using TensorFlow backend.
tensorflow
Data folder: /mnt/batch/tasks/shared/LS_root/jobs/amyboyd-ga-ws/azureml/deeplearning_fashion_1544876526872/mounts/workspaceblobstore/keras-fashion

32768/29515 [=================================] - 0s 0us/step

26427392/26421880 [==============================] - 0s 0us/step

8192/5148 [===============================================] - 0s 0us/step

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_____________________________________________________

57600/60000 [===========================>..] - ETA: 0s - loss: 0.3384 - acc: 0.8799


60000/60000 [==============================] - 20s 339us/step - loss: 0.3388 - acc: 0.8796 - val_loss: 0.2942 - val_acc: 0.8938
Epoch 5/24

20096/60000 [=========>....................] - ETA: 13s - loss: 0.3191 - acc: 0.8828


46336/60000 [======================>.......] - ETA: 4s - loss: 0.3182 - acc: 0.8839


60000/60000 [==============================] - 21s 345us/step - loss: 0.3162 - acc: 0.8851 - val_loss: 0.2805 - val_acc: 0.8978
Epoch 6/24

16000/60000 [=======>......................] - ETA: 14s - loss: 0.2956 - acc: 0.8925


...

48512/60000 [=======================>......] - ETA: 3s - loss: 0.1746 - acc: 0.9320


60000/60000 [==============================] - 20s 340us/step - loss: 0.1762 - acc: 0.9319 - val_loss: 0.2366 - val_acc: 0.9165
Epoch 24/24

18560/60000 [========>.....................] - ETA: 13s - loss: 0.1676 - acc: 0.9357


44416/60000 [=====================>........] - ETA: 5s - loss: 0.1686 - acc: 0.9359


60000/60000 [==============================] - 21s 344us/step - loss: 0.1708 - acc: 0.9349 - val_loss: 0.2352 - val_acc: 0.9193
Predict the test set
Test Loss:  0.23518497274518013
Test Accuracy:  0.9193
Accuracy is 0.9193


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service

Execution Summary
RunId: deeplearning_fashion_1544876526872



{'runId': 'deeplearning_fashion_1544876526872',
 'target': 'gpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-15T12:27:35.176939Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'd3f0f76d-c2f2-4818-aa87-b8ae13787ecd'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': ['--data-folder', '$AZUREML_DATAREFERENCE_workspaceblobstore'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'gpucluster',
  'DataReferences': {'workspaceblobstore': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['azureml-defaults']},


## Evaluate the model output

In [16]:
print(run.get_metrics())

{'accuracy': 0.9193}


In [17]:
print(run.get_file_names())

['azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'outputs/model.h5', 'driver_log', 'azureml-logs/azureml.log']


## Are you happy with the model??? Register it in Azure Machine Learning to manage

In [19]:
# register model 
model = run.register_model(model_name='keras_dl_fashion_test', model_path='outputs/')
print(model.name, model.id, model.version, sep = '\t')

Deprecated, use RunHistoryFacade.assets instead.


keras_dl_fashion_test	keras_dl_fashion_test:2	2


In [ ]:
# If you are using Azure Machine Learning managed compute it will automatically deallocte to zero nodes
# if not remember to spin down your compute targets to save money